In [2]:
from MyDataset import MyDataset

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

# ConvLSTM Cell

In [3]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))

# ConvLSTM

In [4]:
class ConvLSTMEncoder(nn.Module):

    """
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.
    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTMEncoder, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [5]:
class ConvLSTMDecoder(nn.Module):

    """
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.
    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTMDecoder, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                if t+1 != seq_len:
                    cur_layer_input[0][t+1] = h
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

# Sequence to Sequence

In [6]:
class SequenceToSequence(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers):
        super(SequenceToSequence, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        
        self.convLSTM_Encoder = ConvLSTMEncoder(self.input_dim,
                                    hidden_dim=self.hidden_dim,
                                    kernel_size=self.kernel_size,
                                    num_layers=self.num_layers
                                   ).to(device)
        self.convLSTM_Decoder = ConvLSTMDecoder(self.input_dim,
                                    hidden_dim=self.hidden_dim,
                                    kernel_size=self.kernel_size,
                                    num_layers=self.num_layers
                                   ).to(device)
        
    def forward(self, input_tensor, hidden_state=None):
        input_decoder = torch.zeros_like(input_tensor)
        # layer_output_list: h, last_state_list: h&c
        h1, hc1 = self.convLSTM_Encoder(input_tensor)
        input_decoder[0][0] = hc1[0][0][-1]
        h2, hc2 = self.convLSTM_Decoder(input_decoder)
        
        return h2, hc2

# Train

In [14]:
# path
DATA_DIR = '../Innoserve/Data/2019_A_heat_map/'
# model
CUDA_DEVICE_IDEX = 1
BATCH_SIZE = 1
LEARNING_RATE = 1e-2
EPOCHES = 5
# data loader
INPUT_SEQ_LEN = 12
OUTPUT_SEQ_LEN = 12
PIN_MEMORY = True
NUM_WORKERS = 2

In [8]:
torch.manual_seed(0)
device = torch.device("cuda:"+str(CUDA_DEVICE_IDEX) if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

Device: cuda:1


In [10]:
transform_train = torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[2], std=[0.5])])
transform_target = torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[2], std=[0.5])])

dataset = MyDataset(dir=DATA_DIR,
                    history_window=INPUT_SEQ_LEN,
                    future_window=OUTPUT_SEQ_LEN,
                    transform=transform_train,
                    target_transform=transform_target
                   )
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=BATCH_SIZE, 
                                          drop_last=True, 
                                          num_workers=NUM_WORKERS,
                                          pin_memory=PIN_MEMORY)

In [15]:
model = SequenceToSequence(input_dim=1,
                           hidden_dim=[1],
                           kernel_size=(3, 3),
                           num_layers=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, nesterov=True)
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [12]:
for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    layer_output_list, last_state_list = model(x)
    print('x: \t\t\t', x.shape)
    print('y[0]: \t\t\t', y[0].shape)
    print('layer_output_list[0]: \t', layer_output_list[0].shape)
    print('last_state_list[0][0]: \t', last_state_list[0][0].shape)
    break

x: 			 torch.Size([1, 12, 1, 49, 41])
y[0]: 			 torch.Size([12, 1, 49, 41])
layer_output_list[0]: 	 torch.Size([12, 1, 1, 49, 41])
last_state_list[0][0]: 	 torch.Size([12, 1, 49, 41])


In [16]:
for epoch in range(EPOCHES):
    running_loss = 0.0
    for i, data in enumerate(data_loader):
        
        x, y = data
#         x = x / 20714
#         y = y / 20714
        x = x.to(device)
        y = y.to(device)
        # Forwarding
        _, last_state_list = model(x)
#         y_pred = y_pred * 20714
#         loss = criterion(y_pred[0][0], y)
        loss = torch.sqrt(criterion(last_state_list[0][0], y[0]))
        optimizer.zero_grad()
#         # Backpropagation
        loss.backward()
        optimizer.step()
        running_loss += float(loss.item())
#         print('i: {}\tloss:{}'.format(i, loss.item()))
    print('epoch {}/{}\tTrain loss: {:.4f}'.format(epoch+1, EPOCHES, running_loss/(i+1)))

epoch 1/5	Train loss: 1430.7215
epoch 2/5	Train loss: 1430.6902
epoch 3/5	Train loss: 1430.6884
epoch 4/5	Train loss: 1430.6877
epoch 5/5	Train loss: 1430.6873
